In [1]:
import tensorflow as tf
from google.colab import drive

In [2]:
drive.mount('/content/drive')
model_save_path = "/content/drive/MyDrive/best_model_EffB0.keras"
!cp $model_save_path "model_effb0.keras"

Mounted at /content/drive


In [ ]:
# load model
model = tf.keras.models.load_model("model_effb0.keras")
# save in tf saved model format
saved_model_dir = "/content/savedModel"
tf.saved_model.save(model, saved_model_dir)

In [ ]:
!zip -r tflite_mode.zip savedModel

  adding: savedModel/ (stored 0%)
  adding: savedModel/variables/ (stored 0%)
  adding: savedModel/variables/variables.index (deflated 77%)
  adding: savedModel/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: savedModel/fingerprint.pb (stored 0%)
  adding: savedModel/saved_model.pb (deflated 91%)
  adding: savedModel/assets/ (stored 0%)


In [ ]:
# create label.txt.
label_map_reversed = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
label_map = {value: key for key, value in label_map_reversed.items()}

f = open('labels.txt', 'a')
for i in range(7):
  f.write(label_map[i]+"\n")
f.close()

In [ ]:
tflite_model_path = "model_effb0.tflite"
# Converting a tf.Keras model to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Converting a saved model to a TensorFlow Lite model.
#converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the model.
with open(tflite_model_path, 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmp7xv5vtra'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  136385325707184: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136383595947536: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136383585754224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383585755808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383585762144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383585758976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383585761264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383542003488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383542003312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136383542007008: TensorSpec(shape=(), dtype=tf.resource, name=None)

# Adding metadata
tflite-support throws an error with current tf version -> downgrade to tf 2.13

In [ ]:
# tflite-support throws an error with newest tf version
# downgrade tf for tflite-support
!pip install tflite-support
!pip install tensorflow=="2.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras

In [ ]:
import os
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

""" ... """
"""Creates the metadata for an image classifier."""

# tflite model and label path
model_file = "model_effb0.tflite"
labels_file = "labels.txt"

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "Skin cancer classifier"
model_meta.description = ("Identify 7 lesion types.")
model_meta.version = "v1"
model_meta.author = "TensorFlow"
model_meta.license = ("Apache License. Version 2.0 "
                      "http://www.apache.org/licenses/LICENSE-2.0.")


# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()

input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(224, 224))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the 7 labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(labels_file)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

# write metadata to .tflite file
populator = _metadata.MetadataPopulator.with_model_file(model_file)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([labels_file])
populator.populate()

## Display metadata

In [ ]:
displayer = _metadata.MetadataDisplayer.with_model_file(model_file)
print("Metadata populated:")
print(displayer.get_metadata_json())

print("Associated file(s) populated:")
for file_name in displayer.get_packed_associated_file_list():
  print("file name: ", file_name)
  print("file content:")
  print(displayer.get_associated_file_buffer(file_name))

Metadata populated:
{
  "name": "Skin cancer classifier",
  "description": "Identify 7 lesion types.",
  "version": "v1",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified. The expected image is 224 x 224, with three channels (red, blue, and green) per pixel. Each value in the tensor is a single byte between 0 and 255.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              255.0
            ],
            

In [ ]:
import numpy as np
class_names = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]
sunflower_url = "https://www.hautarzt-kronberg.de/templates/yootheme/cache/f9/Spinaliom-f9268270.jpeg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
for p in predictions[0]:
  print(p)
score = tf.nn.softmax(predictions[0])


print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
0.0
0.0
0.0
0.0
0.0
0.0
1.0
This image most likely belongs to vasc with a 31.18 percent confidence.
